In [ ]:
# Import necessary packages
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.image 
import matplotlib.patches
import dlib
import numpy
import pandas as pd
from nd2reader import ND2Reader
import pims
import multiprocessing
from joblib import Parallel, delayed
from pathlib import Path

In [ ]:
# Define the top level directory containing subdirectories for each tissue 
main_folder = ''

# Create a list of subdirectories 
file_list = glob.glob(main_folder + "*.nd2")

# Import .csv file with bounding box coordinates
# Make sure this file is in the main folder with the name "Coordinates.csv"
coords_list = pd.read_csv(main_folder + "Coordinates.csv")
coords_list = coords_list.set_index('Tissue')

In [ ]:
# Define a class for tracking a pillar head
class pillarTrack: 
    def __init__(self, video_path, bounding_box): 
        # bounding_box must be a dlib.rectangle object! 
        self.frame_list = None
        self.x_values  = None
        self.y_values = None
        self.video_path = video_path
        self.bounding_box = bounding_box
    
    def correlation_tracker(self, left_border, top_border):
        # Initialize correlation tracker object
        tracker = dlib.correlation_tracker()
        
        # List all .jpg files in the binarized video folder
        file_list = sorted(glob.glob(os.path.join(self.video_path, "*jpg")))
        num_files = len(file_list)
        
        video_name = self.video_path.split("\\")[(len(self.video_path.split("\\")))-1]
        parent = Path(self.video_path).parent
        
        # Initialize numpy arrays for the frame list and coordinates of the   center of the pillar head
        self.frame_list = numpy.zeros(num_files)
        self.x_values = numpy.zeros(num_files)
        self.y_values = numpy.zeros(num_files)
        
        # Track frames as they are loaded
        for k, f in enumerate(file_list):
            print("Processing Video {} Frame {}".format(video_name, k))
            img = dlib.load_rgb_image(f)

            # Initialize the tracker on first frame based on pre-defined pillar head location
            if k == 0:
               tracker.start_track(img, self.bounding_box)
           
            # Attempt to track from the previous frame 
            else:
                tracker.update(img)
            
            # Determines the center of the tracked pillar head
            pos = tracker.get_position()
            
            # Finds the heigth, width, and center of the tracked pillar head
            w = dlib.drectangle.width(pos)
            h = dlib.drectangle.height(pos)
            left = dlib.drectangle.left(pos)
            bottom = dlib.drectangle.bottom(pos)
            xmean = left + w/2
            ymean = bottom - h/2
            
            # Adds the coordinates of the pillar head center to the appropriate arrays
            self.frame_list[k] = k
            self.x_values[k] = xmean + left_border
            self.y_values[k] = ymean + top_border
            
        # Exports pillar trace to .csv file 
        export = pd.DataFrame({'Frame': self.frame_list, 'X': self.x_values, 'Y': self.y_values})
            
        CSV_name = os.path.join(parent, video_name + ".csv")
        export.to_csv(CSV_name)



In [ ]:
# Define a class for determining the distance between two pillar heads 
class pillarDistance:
    def __init__(self, left_pillar, right_pillar, frame_rate):
        # left_pillar and right_pillar must be pillarTrack objects
        self.distance = None
        self.displacement = None
        self.times = None 
        self.left_pillar = left_pillar
        self.right_pillar = right_pillar
        self.frame_rate = frame_rate
        
    def displacement_calc(self, s):
        # First check that the two pillarTrack objects are the same size 
        
        # Calculate the distance between the two pillar heads using the 
        x_diff = self.right_pillar.x_values - self.left_pillar.x_values
        y_diff = self.right_pillar.y_values - self.left_pillar.y_values 
        x_diff_sq = numpy.square(x_diff)
        y_diff_sq = numpy.square(y_diff)
        sum_sq = x_diff_sq + y_diff_sq
        self.distance = numpy.sqrt(sum_sq)
        
        # Define the distance between the pillars in the first frame as the baseline distance
        # Displacement will be calculated as change from this baseline distance
        baseline = self.distance[0]
        self.displacement = baseline - self.distance
        
    def plot_displacement(self):
        
        # Convert the frame list to time in seconds 
        
        # Plot the pillar displacement over time 
        plt.plot(self.left_pillar.frame_list, self.displacement, )
    
    

In [ ]:
# Program using the above classes for all .nd2 files in the main folder

# We will now define this as a function (new to version 2)

def generate_trace(current_file): 
    
    # Utilize try, except structure to prevent errors from stopping the code 
    try: 
        name_nd2 = current_file.split("\\")[(len(current_file.split("\\")))-1]
        name = name_nd2[:(len(name_nd2)-4)]

        # Make new directory for .jpgs from selected .nd2 file 
        path = os.path.join(main_folder, name + "_left")

        # Check if directory already exists
        try: 
            os.mkdir(path)
        except: 
            print("Directory for {} Left Pillar Already Exists".format(name))
            pass

        path1 = os.path.join(main_folder, name + "_right")

        # Check if directory already exists
        try: 
            os.mkdir(path1)
        except: 
            print("Directory for {} Right Pillar Already Exists".format(name))
            pass

        # Import .nd2 file 
        video = ND2Reader(current_file)

        # Determine video parameters 
        frame_rate = video.frame_rate
        num_frames = len(video)

        # Start with Left Pillar

        # Determine region to crop 
        buffer = 30
        left_border = int(coords_list.loc[name, 'LP_left']) - buffer
        top_border = int(coords_list.loc[name, 'LP_top']) - buffer
        right_border = int(coords_list.loc[name, 'LP_right']) + buffer 
        bottom_border = int(coords_list.loc[name, 'LP_bottom']) + buffer

        right_crop = video.frame_shape[1] - right_border
        bottom_crop = video.frame_shape[0] - bottom_border

        cropped_video = pims.process.crop(video, ((top_border, bottom_crop), (left_border, right_crop)))


        # Loops through frames, crops, enhances contrast, and saves as .jpg
        for i in range(num_frames):
            frame = cropped_video[i]
            # frame = frame * 256
            save_name = os.path.join(path, name + "_" + str(i).zfill(4) +".jpg")
            plt.imsave(save_name, frame, cmap="gray")

        print("Processing Video {}".format(name))

        # Select coordinates from .csv to put in initial bounding boxes 
        # Start with left pillar bounding box
        left = buffer
        top = buffer
        right = cropped_video.frame_shape[1] - buffer
        bottom = cropped_video.frame_shape[0] - buffer
        left_bounding_box = dlib.rectangle(left, top, right, bottom)


        left_track = pillarTrack(video_path=path, bounding_box=left_bounding_box)
        left_track.correlation_tracker(left_border = left_border, top_border = top_border)

        # Right pillar bounding box
        left_border = int(coords_list.loc[name, 'RP_left']) - buffer
        top_border = int(coords_list.loc[name, 'RP_top']) - buffer
        right_border = int(coords_list.loc[name, 'RP_right']) + buffer
        bottom_border = int(coords_list.loc[name, 'RP_bottom']) + buffer

        right_crop = video.frame_shape[1] - right_border
        bottom_crop = video.frame_shape[0] - bottom_border

        cropped_video = pims.process.crop(video, ((top_border, bottom_crop), (left_border, right_crop)))


        # Loops through frames, crops, enhances contrast, and saves as .jpg
        for i in range(num_frames):
            frame = cropped_video[i]
            #frame = frame * 256
            save_name = os.path.join(path1, name + "_" + str(i).zfill(4) +".jpg")
            plt.imsave(save_name, frame, cmap="gray")

        print("Processing Video {}".format(name))

        # Select coordinates from .csv to put in initial bounding boxes 
        # Start with left pillar bounding box
        left = buffer
        top = buffer
        right = cropped_video.frame_shape[1] - buffer
        bottom = cropped_video.frame_shape[0] - buffer

        right_bounding_box = dlib.rectangle(left, top, right, bottom)

        right_track = pillarTrack(video_path=path1, bounding_box=right_bounding_box)
        right_track.correlation_tracker(left_border = left_border, top_border = top_border)
    except: 
        pass
     


In [ ]:
#Use multiprocessing to run the generate_trace function in paraellel for all videos in the main directory

Parallel(n_jobs=-2, verbose=10)(delayed(generate_trace)(i) for i in file_list)